## Setting up the environment

In [1]:
!python3 -m pip install hydra-core==1.1

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!python3 -m pip install scipy==1.7.3

BRANCH = 'main'
# If you're using Google Colab and not running locally, uncomment and run this cell.
!apt-get install sox libsndfile1 ffmpeg
!python -m pip install wget unidecode pynini==2.1.4
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

#=== load the repo and data (Thanks Synthbot) ===
!sudo dpkg --configure -a
!sudo apt -qq install -y sox
!git clone "https://github.com/synthbot-anon/synthbot.git" /content/synthbot
!(cd /content/synthbot; git checkout experimental)
!python3 -m pip install pysoundfile 
!python3 -m pip install librosa

!jupyter nbextension enable --py --sys-prefix widgetsnbextension

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.25-10ubuntu0.16.04.3).
ffmpeg is already the newest version (7:2.8.17-0ubuntu0.1).
sox is already the newest version (14.4.1-5+deb8u4ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
sox is already the newest version (14.4.1-5+deb8u4ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
fatal: destination path '/content/synthbot' already exists and is not an empty directory.
Already on 'experimental'
Your branch is up-to-date with 'origin/experimental'.
You should consider upgrading via the '/usr/local/bin/python3 -m 

In [4]:
home_path = !(echo $HOME)
home_path = home_path[0]
print(home_path)

/root


In [7]:
#downloading the dataset
id='15xGE5zFLm6hLOk6PNE_QHMcEUTTSZ8Il'
character='spike'
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={id}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={id}" -O {character+'.tar'} && rm -rf /tmp/cookies.txt


--2022-07-23 10:18:30--  https://docs.google.com/uc?export=download&confirm=t&id=15xGE5zFLm6hLOk6PNE_QHMcEUTTSZ8Il
Resolving docs.google.com (docs.google.com)... 142.251.2.102, 142.251.2.138, 142.251.2.100, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ue3sp5t64btclbth24810q6b9oj56en8/1658571450000/05434657448262868821/*/15xGE5zFLm6hLOk6PNE_QHMcEUTTSZ8Il?e=download&uuid=3374bb2e-9e9f-4cf7-b5b4-f54985bb2652 [following]
--2022-07-23 10:18:30--  https://doc-10-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ue3sp5t64btclbth24810q6b9oj56en8/1658571450000/05434657448262868821/*/15xGE5zFLm6hLOk6PNE_QHMcEUTTSZ8Il?e=download&uuid=3374bb2e-9e9f-4cf7-b5b4-f54985bb2652
Resolving doc-10-84-docs.googleusercontent.com (doc-10-84-docs.googleusercontent.com)... 142.251.2.132, 2607:

In [15]:
skip_noisy = True # Disable to train with Noisy Data included
allowed_emotions = """
Anxious
Angry
Annoyed
Amused
Confused
Crazy
Disgust
Exhausted
Fear
Happy
Neutral
Sad
Serious
Singing
Shouting
Surprised
Smug
Love
Sarcastic
Tired
Whispering
Whining
""".split("\n")[1:-1]

percentage_training_data = 0.95 # 90% of Data will be used for training, 5% for Validation

##############################################################################################

import sys
sys.path.append('/content/synthbot/src')
from ponysynth.corpus import ClipperArchive, phoneme_transcription
import librosa
import subprocess

archive_fn = "./"+character+".tar"
archive = ClipperArchive(archive_fn)

data_path = character
allowed_emotions = [x.lower() for x in allowed_emotions]
!mkdir {data_path}
!mkdir {data_path+"/out"}
all_clips = []; all_clips_arpa = []; skipped_count=0; too_short_count=0; emotion_skip=0
for key in archive.keys(): # write the audio files for processing in bash terminal
  audio = archive.read_audio(key)
  audio_fn = '{}/{}.wav'.format(data_path, key)
  audio_fn_ = '{}/{}.wav'.format(data_path+"/out", key)
  with open(audio_fn, 'wb') as audio_out:
    audio_out.write(audio.read())

In [16]:
%%script bash
# trim all 48Khz files
cd spike
for input in *.wav; do
  output="out/$input"
  sox "$input" "$output" silence 1 0.05 0.1% reverse silence 1 0.05 0.1% reverse;
done

In [17]:
import os

should_continue = 0 # should run "continue" command inside outer loop.
for key in archive.keys():
  label = archive.read_label(key)
  audio_fn = '{}/{}.wav'.format(data_path, key)
  audio_fn_ = '{}/{}.wav'.format(data_path+"/out", key)
  if (label['noise'] in ['Very Noisy','Noisy']) and skip_noisy: os.remove(audio_fn_); skipped_count+=1; continue

  for tag in label['tags']:
     if tag.lower() not in allowed_emotions:
       try: os.remove(audio_fn_)
       except: pass
       print(tag+" emotion not in list"); emotion_skip+=1; should_continue = 1; break # this is supposed to break the outer loop
  if should_continue: should_continue = 0; continue

  audio = archive.read_audio(key)
  transcript = label['utterance']['content']
  
  if os.stat(audio_fn_).st_size < 71602:
    #print("Skipping Audio, Duration: "+str(len(librosa.core.load(audio_fn_, sr=48000)[0])/48000))
    try: os.remove(audio_fn_)
    except: pass
    too_short_count+=1; continue # Skips files based on size.

  filelist_line = "{}|{}".format(audio_fn_, transcript)
  all_clips.append(filelist_line)
  filelist_line = "{}|{}".format(audio_fn_, phoneme_transcription(label))
  all_clips_arpa.append(filelist_line)
    
print(str(skipped_count)+" Files are too Noisy.")
print(str(emotion_skip)+" Files contain an emotion not in permitted emotions")
print(str(too_short_count)+" Files are too short")
print(str(len(list(archive.keys()))-(skipped_count+too_short_count+emotion_skip))+" Files kept in dataset.")


####################################################################################################################
# shuffle the training data
import random
random.seed(0)
random.shuffle(all_clips)

# get train, test, validation splits
num_clips = len(all_clips)
train_end = int(num_clips * percentage_training_data)

train = all_clips[:train_end]
validation = all_clips[train_end:]

train_arpa = all_clips_arpa[:train_end]
validation_arpa = all_clips_arpa[train_end:]

# dump the info to filelist files
with open('train_filelist.txt', 'w') as train_out:
  train_out.write('\n'.join(train)+'\n'+'\n'.join(train_arpa)+"")
with open('val_filelist.txt', 'w') as val_out:
  val_out.write('\n'.join(validation)+'\n'+'\n'.join(validation_arpa)+"")

##################################################################################################################

#imports
import os
import json
import random
import librosa

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from nemo.collections.tts.models import FastPitchModel

######################################################################################

#creating the json files for training and validation
with open('./train_filelist.txt') as f :
  records = f.readlines()

print("Number of records : ",len(records))

train_manifest = 'fastpitch_train.json'

train_rec = []
random.shuffle(records)
count = 0

for i in records :
  #if count > 1200 :
  #  break
  
  i = i.split('|')
  audio_filepath = i[0]
  text = i[-1].strip('\n')
  if '{' in text :
    #print(text)
    continue
  count = count + 1

  duration = librosa.get_duration(filename=audio_filepath)
  r = {
       "audio_filepath" : audio_filepath,
       "text" : text,
       "duration" : round(duration,1),
       "text_no_preprocessing" : text
    }

  train_rec.append(r)

with open(train_manifest, "w") as f:
    for s in train_rec:
        f.write(json.dumps(s) + '\n')
        
print("Training Data : ", len(train_rec))

##########################################################################################

with open('./val_filelist.txt') as f :
  records_val = f.readlines()

print("Number of records : ", len(records_val))

val_manifest = 'fastpitch_val.json'
count = 0
val_rec = []
random.shuffle(records_val)

for i in records_val:
  #if count > 50 :
  #  break
    
  i = i.split('|')
  audio_filepath = i[0]
  text = i[-1].strip('\n')

  if '{' in text :
    #print(text)
    continue
  count = count + 1
  duration = librosa.get_duration(filename=audio_filepath)
  r = {
       "audio_filepath" : audio_filepath,
       "text" : text,
       "duration" : round(duration,1),
       "text_no_preprocessing" : text
    }

  val_rec.append(r)

print(len(val_rec))
with open(val_manifest, "w") as f:
    for s in val_rec:
        f.write(json.dumps(s) + '\n')

####################################################################################

FastPitchModel.from_pretrained("tts_en_fastpitch")
nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_en_fastpitch_align.nemo")]
print(f"Copying {nemo_files[0]} to ./")
Path("./tts_en_fastpitch_align.nemo").write_bytes(nemo_files[0].read_bytes())

###################################################################################

!wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/fastpitch_finetune.py

!mkdir -p conf \
&& cd conf \
&& wget https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/conf/fastpitch_align_v1.05.yaml \
&& cd ..

#######################################
# additional files
!mkdir -p tts_dataset_files && cd tts_dataset_files \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/cmudict-0.7b_nv22.07 \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/heteronyms-030921 \
&& wget wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/nemo_text_processing/text_normalization/en/data/whitelist/lj_speech.tsv \
&& cd ..


2586 Files are too Noisy.
0 Files contain an emotion not in permitted emotions
177 Files are too short
1461 Files kept in dataset.
Number of records :  2774
Training Data :  1387
Number of records :  148
74
[NeMo I 2022-07-23 10:23:22 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.10.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo.
[NeMo I 2022-07-23 10:23:22 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.10.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo
[NeMo I 2022-07-23 10:23:22 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-07-23 10:23:27 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-23 10:23:58 experimental:28] Module <class 'nemo.collections.tts.torch.g2ps.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-07-23 10:23:59 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-23 10:23:59 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matri

[NeMo I 2022-07-23 10:23:59 features:200] PADDING: 1
[NeMo I 2022-07-23 10:24:17 save_restore_connector:243] Model FastPitchModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.10.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo.
Copying /root/.cache/torch/NeMo/NeMo_1.10.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo to ./
--2022-07-23 10:24:18--  https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/fastpitch_finetune.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1751 (1.7K) [text/plain]
Saving to: ‘fastpitch_finetune.py’

fastpitch_finetune. 100%[===================>]   1.71K  --.-KB/s    in 0s      

2022-07-23 10:24:18 (32.2 MB/s) - ‘fastpitch_

In [18]:
from nemo.collections.tts.torch.g2ps import EnglishG2p
from nemo.collections.tts.torch.data import TTSDataset
from nemo_text_processing.text_normalization.normalize import Normalizer
from nemo.collections.tts.torch.tts_tokenizers import EnglishPhonemesTokenizer, EnglishCharsTokenizer

import torch
from tqdm.notebook import tqdm


# Text normalizer
text_normalizer = Normalizer(
    lang="en", 
    input_case="cased", 
    whitelist="./tts_dataset_files/lj_speech.tsv"
)

text_normalizer_call_kwargs = {
    "punct_pre_process": True,
    "punct_post_process": True
}

# Text tokenizer
text_tokenizer = EnglishCharsTokenizer()

def pre_calculate_supplementary_data(sup_data_path, sup_data_types, text_tokenizer, text_normalizer, text_normalizer_call_kwargs):
    # init train and val dataloaders
    stages = ["train", "val"]
    stage2dl = {}
    for stage in stages:
        ds = TTSDataset(
            manifest_filepath=f"fastpitch_{stage}.json",
            sample_rate=16000,
            sup_data_path=sup_data_path,
            sup_data_types=sup_data_types,
            n_fft=1024,
            win_length=1024,
            hop_length=256,
            window="hann",
            n_mels=80,
            lowfreq=0,
            highfreq=8000,
            text_tokenizer=text_tokenizer,
            text_normalizer=text_normalizer,
            text_normalizer_call_kwargs=text_normalizer_call_kwargs

        ) 
        stage2dl[stage] = torch.utils.data.DataLoader(ds, batch_size=1, collate_fn=ds._collate_fn, num_workers=0)

    # iteration over dataloaders
    pitch_mean, pitch_std, pitch_min, pitch_max = None, None, None, None
    for stage, dl in stage2dl.items():
        pitch_list = []
        print(stage)
        print(dl)
        for batch in tqdm(dl, total=len(dl)):
            tokens, tokens_lengths, audios, audio_lengths, attn_prior, pitches, pitches_lengths = batch
            pitch = pitches.squeeze(0)
            pitch_list.append(pitch[pitch != 0])

        if stage == "train":
            pitch_tensor = torch.cat(pitch_list)
            pitch_mean, pitch_std = pitch_tensor.mean().item(), pitch_tensor.std().item()
            pitch_min, pitch_max = pitch_tensor.min().item(), pitch_tensor.max().item()
    
    print("Pitch_mean : ",pitch_mean)
    print("Pitch_std : ",pitch_std)
    print("Pitch_min : ",pitch_min)
    print("Pitch_max : ",pitch_max)
            
    return pitch_mean, pitch_std, pitch_min, pitch_max

fastpitch_sup_data_path = "fastpitch_sup_data_folder"
sup_data_types = ["align_prior_matrix", "pitch"]

pitch_mean, pitch_std, pitch_min, pitch_max = pre_calculate_supplementary_data(
    fastpitch_sup_data_path, sup_data_types, text_tokenizer, text_normalizer, text_normalizer_call_kwargs
)

[NeMo I 2022-07-23 10:24:23 tokenize_and_classify:87] Creating ClassifyFst grammars.
[NeMo I 2022-07-23 10:24:52 data:186] Loading dataset from fastpitch_train.json.


1387it [00:19, 69.53it/s]

[NeMo I 2022-07-23 10:25:12 data:221] Loaded dataset with 1387 files.
[NeMo I 2022-07-23 10:25:12 data:223] Dataset contains 0.82 hours.
[NeMo I 2022-07-23 10:25:12 data:315] Pruned 0 files. Final dataset contains 1387 files
[NeMo I 2022-07-23 10:25:12 data:318] Pruned 0.00 hours. Final dataset contains 0.82 hours.
[NeMo I 2022-07-23 10:25:12 data:186] Loading dataset from fastpitch_val.json.



74it [00:01, 73.92it/s]

[NeMo I 2022-07-23 10:25:13 data:221] Loaded dataset with 74 files.
[NeMo I 2022-07-23 10:25:13 data:223] Dataset contains 0.04 hours.
[NeMo I 2022-07-23 10:25:13 data:315] Pruned 0 files. Final dataset contains 74 files
[NeMo I 2022-07-23 10:25:13 data:318] Pruned 0.00 hours. Final dataset contains 0.04 hours.
train


  0%|          | 0/1387 [00:00<?, ?it/s]

val


  0%|          | 0/74 [00:00<?, ?it/s]

Pitch_mean :  1.0
Pitch_std :  0.0
Pitch_min :  1.0
Pitch_max :  1.0


In [ ]:
#setting up wandb credentials
#weights and biases 
wandb_api_key = "d06dcb679ec9cf0d71c81cb750d142b6b90f3d25"
wandb_project_name = 'MLP'
wandb_run_name = character

!wandb login ${wandb_api_key}

In [ ]:
#resuming the training 
#+init_from_ptl_ckpt=Fastpitch_epoch_356.ckpt
# trainer.max_epochs=500 ~trainer.max_epochs 
#+init_from_nemo_model=tts_en_fastpitch_align.nemo \
! (python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=fastpitch_train.json \
  validation_datasets=fastpitch_val.json \
  sup_data_path=fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.07 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=training_logs \
  +init_from_nemo_model=tts_en_fastpitch_align.nemo \
  trainer.max_epochs=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=7 \
  model.train_ds.dataloader_params.batch_size=8 model.validation_ds.dataloader_params.batch_size=8 \
  model.n_speakers=1 model.pitch_mean=324.43 model.pitch_std=112.51 \
  model.pitch_fmin=65.41 model.pitch_fmax=2093 model.optim.lr=2e-4 \
  +exp_manager.create_wandb_logger=true \
  +exp_manager.wandb_logger_kwargs.name=${wandb_run_name} \
  +exp_manager.wandb_logger_kwargs.project=${wandb_project_name} \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

## Checking the checkpoint

In [ ]:
checkpoint = ''

In [ ]:
spec_model = FastPitchModel.load_from_checkpoint(checkpointpath)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

In [ ]:
custom_infer("My money don't jiggle jiggle, it folds, make your body wiggle wiggle !", spec_model2, vocoder)